# RigidBodySim.jl

RigidBodySim is a simulation environment built on top of [RigidBodyDynamics.jl](https://github.com/JuliaRobotics/RigidBodyDynamics.jl), [DifferentialEquations.jl](https://github.com/JuliaDiffEq/DifferentialEquations.jl), and [MeshCat.jl](https://github.com/rdeits/MeshCat.jl).

See the [latest documentation](https://JuliaRobotics.github.io/RigidBodySim.jl/latest) and the [quick start guide](https://nbviewer.jupyter.org/github/JuliaRobotics/RigidBodySim.jl/blob/master/notebooks/Quick%20start%20guide.ipynb) for more information and examples.

# Loading the Atlas `Mechanism`

We'll first create a `RigidBodyDynamics.Mechanism` for the Atlas robot.

In [1]:
# Tell Julia to use the packages specified in the Project.toml
# and Manifest.toml files contained in the DynamicWalking2018.jl
# repository. This ensures that you will have the same packages
# that were used to create this tutorial. 
using Pkg
pkg"activate .."
pkg"instantiate"

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `git@github.com:JuliaRegistries/General.git`


In [2]:
using RigidBodySim
using RigidBodyDynamics
using AtlasRobot # tiny package containing Atlas URDF and meshes

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/41f396c72270c88d4098c6aff59a013a05364eb9-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/41f396c72270c88d4098c6aff59a013a05364eb9-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

┌ Info: Recompiling stale cache file /home/rdeits/.julia/compiled/v1.0/AtlasRobot/GTN3n.ji for AtlasRobot [436ed74b-f7b6-57fe-bd82-efdbd6c0ab0c]
└ @ Base loading.jl:1190


In [3]:
atlas = AtlasRobot.mechanism(floating=false)

Spanning tree:
Vertex: world (root)
  Vertex: ltorso, Edge: back_bkz
    Vertex: mtorso, Edge: back_bky
      Vertex: utorso, Edge: back_bkx
        Vertex: l_clav, Edge: l_arm_shz
          Vertex: l_scap, Edge: l_arm_shx
            Vertex: l_uarm, Edge: l_arm_ely
              Vertex: l_larm, Edge: l_arm_elx
                Vertex: l_ufarm, Edge: l_arm_uwy
                  Vertex: l_lfarm, Edge: l_arm_mwx
                    Vertex: l_hand, Edge: l_arm_lwy
        Vertex: head, Edge: neck_ay
        Vertex: r_clav, Edge: r_arm_shz
          Vertex: r_scap, Edge: r_arm_shx
            Vertex: r_uarm, Edge: r_arm_ely
              Vertex: r_larm, Edge: r_arm_elx
                Vertex: r_ufarm, Edge: r_arm_uwy
                  Vertex: r_lfarm, Edge: r_arm_mwx
                    Vertex: r_hand, Edge: r_arm_lwy
  Vertex: l_uglut, Edge: l_leg_hpz
    Vertex: l_lglut, Edge: l_leg_hpx
      Vertex: l_uleg, Edge: l_leg_hpy
        Vertex: l_lleg, Edge: l_leg_kny
          Vertex: l_talus

In [20]:
const state = MechanismState(atlas)

MechanismState{Float64, Float64, Float64, …}(…)

# Setting up visualization

We'll again use `MeshCat` for visualization.

In [11]:
using MeshCatMechanisms

In [12]:
visuals = URDFVisuals(AtlasRobot.urdfpath();
    package_path = [AtlasRobot.packagepath()]);

In [15]:
gui = GUI(atlas, visuals)
open(gui);

# Closed loop dynamics

Let's simulate Atlas with a very simple controller:

In [16]:
function control!(τ, t, state)
    # just add a bit of joint damping
    τ .= -0.1 .* velocity(state)
end

control! (generic function with 1 method)

In [17]:
closed_loop_dynamics = Dynamics(atlas, control!);

Or you can set up the controller to run at a fixed rate (in terms of simulation time):

In [18]:
# τ = similar(velocity(state))
# Δt = 1e-3
# controller = PeriodicController(similar(velocity(state)), Δt, control!)
# closed_loop_dynamics = Dynamics(atlas, controller);

# Creating an `ODEProblem`

An `ODEProblem` (from the DifferentialEquations.jl ecosystem) represents a to-be-integrated ordinary differential equation (ODE):

In [21]:
zero!(state)
timespan = (0., 5.)
problem = ODEProblem(closed_loop_dynamics, state, timespan)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 5.0)
u0: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

# Integrating the ODE

To integrate the ODE, we'll be using the Tsitouras 5/4 (5th order) variable time step Runge-Kutta method.

In [22]:
integrator = Tsit5()

Tsit5()

To visualize Atlas while simulating, we'll create a callback that will be run periodically during the integration process:

In [24]:
gui_callback = CallbackSet(gui);

Now we can solve the `ODEProblem`:

In [26]:
sol = solve(problem,
    integrator,
    abs_tol = 1e-10,
    dt = 0.05,
    callback = CallbackSet(gui_callback));

We can also visualize the solution after the fact:

In [28]:
setanimation!(gui.visualizer, sol)